<a href="https://colab.research.google.com/github/nicologhielmetti/AN2DL-challenges/blob/master/challenge_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing
The dataset provided for this challenge is composed by xx images of different height and width. In order to obtain a standard shape for all images, a particular function has been used, namely `tf.keras.preprocessing.image.smart_resize()`.

This function resizes images to a specified target size without affecting the aspect ratio. This step is due to the successive batching of all images that requires a common format for all of them.
In order to avoid a loss of information that would have been caused by applying a crop, it has been computed what the max width/height of the images was by implementing `getMaxImageSize(dataset_dir)`.

This function returns the maximum (width,height), considering all the images of the provided dataset.
For what regards data augmentation and the splitting of the data into two distinct sets, namely train and validation, it has been used the following script which assigns a fraction of 30% of the imgaes to the validation set; the rest will be assigned to the training set.

```
train_data_gen = ImageDataGenerator ( rotation_range=10,            
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3, 
                                      horizontal_flip=True,
                                      fill_mode='reflect',
                                      rescale=1. / 255,
                                      validation_split=0.3,  
                                      preproc_funct=smart_resize()  
                                    )
```
Considering that a strict division of the images in subdirectories representing the target classes is required for the functions involved in the creation of augmented images, we implemented a specific function that requires a ```json_definition``` for the subdirectory construction and the path where such images are located:


# Model Design



> Custom Models


1.  Custom Model 1
2.  Custom Model 2
3.  Custom Model 3

A custom model with six convolutional layers to perform feature extraction and five layers used to perform classification has been implemented. Here early stopping, dropout and l2-regularization are used to prevent overfitting. A low drop-out probability has been selected given the presence of different same-aimed techniques in the architecture.

```
self.classifier.add(tf.keras.layers.Dense(units=512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
```




```
self.classifier.add(tf.keras.layers.Dropout(0.2))
```





> Transfer Learning & Fine Tuning

After the realization of custom models, it has been decided to implement the transfer learning procedure, along with fine tuning, in order to exploit and explore the capabilities of features extractors of some popular networks and thus improve the performances on the classification task proposed by this challenge. The selected architecures are VGG16, VGG19 in two different implementations and XCeption. 


1.   VGG_16


```
tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
```
For what concernes VGG16 a single Dense layer with an higher, with respect to VGG19 and XCeption, number of nodes has been selected as top of the network respecting the original architecture. 
```
model.add(tf.keras.layers.Dense(units=512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
```
A short fine tuning procedure has been implemented to keep the ability of the feature extractor of the network non-specific and basically unaltered.

```
finetuning = True

if finetuning:
    freeze_until = 15
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
```


2.   VGG_19

```
tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
```

In VGG19 a three layered architecture with decreasing number of nodes has been selected as the goal was to learn a good combination of the features extracted by the architecure itself.


```
model.add(tf.keras.layers.Dense(units=256, activation='relu'))
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
```
A slower learning rate, with respect to others architectures,was required by this network to reach good performances on the validation set.

Given the length of the network, in order to adapt it to the specific task a long fine tuning procedure has been chosen.



```
freeze_until = 5
```



3.   Xception

```
vgg = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
```

The same motivations for the choice of the three layered architecture and fine tuning depth still apply to the Xception network as well as for the VGG19 one.

# Model Selection
Yet to come
#Results
Yet to come





